Class structures

- Formulas for just calculating financial ratios
- Functions for retrieval/calculation of financial ratios and figures
- Functions for modelling/projections

- Class for instantiating a particular company and calling data off of that company (yahoo finance integration)

numpy

seaborn

matplotlib

scikit-learn


In [420]:
import pandas as pd
from yahoofinancials import YahooFinancials
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

In [378]:
class Formulas(object):
    
    def total_expenses(operating_expenses,interest,tax,depreciation):
        return abs(operating_expenses) + abs(interest) + abs(tax) + abs(depreciation)
    
    def net_income(revenue,total_expenses):
        return abs(revenue)-(abs(total_expenses))
    
    def gross_profit(revenue,operating_expenses):
        return abs(revenue)-(abs(operating_expenses))
    
    def earnings_per_share(net_income,outstanding_shares):
        return abs(net_income)/abs(outstanding_shares)
    
    def pe_ratio(share_price,earnings_per_share):
        return abs(share_price)/abs(earnings_per_share)
    
    def enterprise_value(market_value,liquid_assets,total_debt):
        return abs(market_value) - abs(liquid_assets) + abs(total_debt)
    
    def ebitda(net_income,tax,interest,depreciation,amortization):
        return abs(net_income)+abs(tax)+abs(interest)+abs(depreciation)+abs(amortization)
    
    def ebit(revenue,cost_of_goods_sold,operating_expenses,depreciation,amortization):
        return abs(revenue) - abs(cost_of_goods_sold) - abs(operating_expenses) - abs(depreciation) - abs(amortization)
    
    def quick_ratio(liquid_assets,receivables,liabilities):
        return (abs(liquid_assets)+abs(receivables))/abs(liabilities)
    
    def income_continuing_operations_margin(ebit,revenue):
        return abs(ebit)/abs(revenue)
    
    def net_margin(net_income,revenue):
        return abs(net_income)/abs(revenue)
    
    def return_on_assets(net_income,beginning_total_assets,ending_total_assets):
        return abs(net_income)/((abs(beginning_total_assets)+abs(ending_total_assets))/2)
    
    def return_on_capital(ebit,corporate_tax_rate,beginning_total_capital,ending_total_capital):
        return (abs(ebit)*(1-abs(corporate_tax_rate)))/((abs(beginning_total_capital)+abs(ending_total_capital))/2)
    
    def return_on_equity(income_continuing_operations,total_equity):
        return abs(income_continuing_operations)/abs(total_equity)
    
    def growth_rate(beginning_figure,ending_figure):
        return (abs(ending_figure)-abs(beginning_figure))/abs(beginning_figure)
    
    def cash_flow_operations(revenue,operating_expenses):
        return abs(revenue-operating_expenses)
    
    def free_cash_flow(cash_flow_operations,interest,corporate_tax_rate,capital_expenditures):
        return abs(cash_flow_operations) + (abs(interest)*(1-abs(corporate_tax_rate))) - abs(capital_expenditures)
    
    def beta(risk_free_ror,security_ror,market_ror):
        return (abs(stocks_ror)-abs(risk_free_ror))/(abs(market_ror)-abs(risk_free_ror))
    
    def cost_of_equity(risk_free_ror, beta, market_ror):
        return abs(risk_free_ror)+(abs(beta)*(abs(market_ror)-abs(risk_free_ror)))
    
    def gross_profit_margin(revenue, cost_of_goods_sold):
        return (abs(revenue)-abs(cost_of_goods_sold))/abs(revenue)
    
    def operating_profit_margin(ebit,revenue):
        return abs(ebit)/abs(revenue)
    
    def net_profit_margin(net_income,revenue):
        return abs(net_income)/abs(revenue)

In [438]:
class figureRetrieval(object):
    
    def __init__(self,ticker):
        self.ticker = ticker.upper()
        self.financials = YahooFinancials(ticker)
        self.corporate_tax_rate = 0.21
        self.risk_free_ror = 0.025
        self.market_ror = 0.098
        self.timeframe = 'annual'
        self.balance_sheet = None
        self.income_statement = None
        self.cash_flow_statement = None
        self.key_statistics = None
        self.historical_eps = None

    def __set_balance_sheet(self):
        if self.balance_sheet == None:
            try:
                self.balance_sheet = self.financials.get_financial_stmts(self.timeframe,'balance')['balanceSheetHistory'][self.ticker]
                self.balance_sheet = self.__clean_statement_timestamp(self.balance_sheet)
            except:
                return "Error: Could not retrieve balance sheet"
    
    def __set_income_statement(self):
        if self.income_statement == None:
            try:
                self.income_statement = self.income_statement = self.financials.get_financial_stmts(self.timeframe,'income')['incomeStatementHistory'][self.ticker]
                self.income_statement = self.__clean_statement_timestamp(self.income_statement)
            except:
                return "Error: Could not retrieve income statement"
    
    def __set_cash_flow_statement(self):
        if self.cash_flow_statement == None:
            try:
                self.cash_flow_statement = self.cash_flow_statement = self.financials.get_financial_stmts(self.timeframe,'cash')['cashflowStatementHistory'][self.ticker]
                self.cash_flow_statement = self.__clean_statement_timestamp(self.cash_flow_statement)
            except:
                return "Error: Could not retrieve cash flow statement"
            
    def __set_key_statistics(self):
        if self.key_statistics == None:
            try:
                self.key_statistics = self.financials.get_key_statistics_data()
            except:
                return "Error: Could not retrieve key statistics"
    
    def __set_historical_eps(self):
        if self.historical_eps == None:
            url = 'https://www.nasdaq.com/earnings/report/{}'.format(self.ticker)
            page = requests.get(url)
            if page.status_code == 200:
                soup = BeautifulSoup(page.text,'html.parser')
                eps_list = []
                td = soup.find_all('td')
                counter = 0
                for i in td[2::5]:
                    try:
                        test = float(i.get_text())
                        counter += 1
                    except:
                        break
                for i in range(counter*5): 
                    eps_list.append(list(soup.find_all('td'))[i].get_text())
                self.historical_eps = pd.DataFrame({'textdate':eps_list[0::5],'timestamp':eps_list[1::5],'eps':eps_list[2::5],'consensus_eps':eps_list[3::5],'surprise':eps_list[4::5]})
                self.historical_eps = self.historical_eps.iloc[::-1]
                self.historical_eps.reset_index(inplace=True,drop=True)
            else:
                return "Error: Could not connect to NASDAQ website"
    
    def __clean_statement_timestamp(self,statement):
        for i in statement:
                for j in i.keys():
                    i[j[:4]] = i.pop(j)
        return statement
    
    def __get_specific_year(self,statement,year):
        for i in statement:
            if str(list(i.keys())[0]) == str(year):
                return i['{}'.format(year)]

    def get_balance_sheet(self):
        self.__set_balance_sheet()
        columns = []
        for i in self.balance_sheet:
            for j in i.keys():
                columns.append(j)
        df = pd.DataFrame(self.balance_sheet[0])
        for i in range(len(columns)):
            df[columns[i]] = pd.DataFrame(self.balance_sheet[i])
        df = df[df.columns[::-1]]
        return df
    
    def get_income_statement(self):
        pass
    
    def get_cash_flow_statement(self):
        pass
    
    def get_key_statistics(self):
        pass
    
    def get_historical_eps(self):
        self.__set_historical_eps()
        return self.historical_eps
    
    def check_ticker(self):
        return self.ticker
    
    def set_corporate_tax_rate(self,corporate_tax_rate):
        self.corporate_tax_rate = corporate_tax_rate
        
    def set_risk_free_ror(self,risk_free_ror):
        self.risk_free_ror = risk_free_ror
        
    def set_market_ror(self,market_ror):
        self.market_ror = market_ror
    

    def revenue(self,year):
        self.__set_income_statement()
        try:
            return self.__get_specific_year(self.income_statement,year)['totalRevenue']
        except:
            return None

    def total_expenses(self,year):
        self.__set_income_statement()
        self.__set_cash_flow_statement()
        try:
            operating_expenses = self.__get_specific_year(self.income_statement,year)['totalOperatingExpenses']
            interest = self.__get_specific_year(self.income_statement,year)['interestExpense']
            tax = self.__get_specific_year(self.income_statement,year)['incomeTaxExpense']
            depreciation = self.__get_specific_year(self.cash_flow_statement,year)['depreciation']
            return Formulas.total_expenses(operating_expenses,interest,tax,depreciation)
        except:
            return None
            
    
    def operating_expenses(self,year):
        self.__set_cash_flow_statement
        try:
            return self.__get_specific_year(self.cash_flow_statement,year)['totalOperatingExpenses']
        except:
            return None
    
    def outstanding_shares(self):
        if self.key_statistics == None:
            pass
    
    def share_price(self):
        pass
    
    def market_value(self):
        pass
    
    def liquid_assets(self):
        pass
    
    def total_debt(self):
        pass
    
    def total_debt(self):
        pass
    
    def tax(self):
        return Formulas.gross_profit(self.revenue(),self.operating_expenses()) * (1-self.corporate_tax_rate)
    
    def interest(self):
        pass
    
    def depreciation(self):
        pass
    
    def amortization(self):
        pass
    
    def cost_of_goods_sold(self):
        pass
    
    def receivables(self):
        pass
    
    def liabilities(self):
        pass
    
    def total_assets(self):
        pass
    
    def total_capital(self):
        pass
    
    def total_equity(self):
        pass
    
    def capital_expenditures(self):
        pass


In [439]:
f = figureRetrieval('cvx')

In [440]:
f.get_historical_eps()

,textdate,timestamp,eps,consensus_eps,surprise
0,Jun2018,07/27/2018,1.78,2.06,-13.59
1,Sep2018,11/02/2018,2.11,2.06,2.43
2,Dec2018,02/01/2019,2.06,1.87,10.16
3,Mar2019,04/26/2019,1.39,1.26,10.32


In [397]:
f.revenue(2018)

265595000000

In [370]:
f.check_ticker()

'CVX'

In [ ]:
class PriceData(object):
    pass

In [ ]:
class RatioCalculation(object):
    # be sure to include period type (quarterly or yearly)
    # and what specific quarter/year

In [ ]:
class TrendModeling(object):
    # this class should model line of best fit trends on first level,
    # and second level acceleration line of best fit trends

For modelling, use a system wherby you can freely adjust the growth parameters, and if you so choose - can use historical growth/acceleration metrics to adjust those parameters for you automatically

chapter 12 should be for modelling 

In [ ]:
class ValueModeling(object):
    

In [ ]:
class RiskModeling(object):
    

In [412]:
# Visualization class
class Visualize:
    #df = df.T.iloc[::-1]
    pass

In [ ]:
class Recommend(object):
    
    def __init__(self,company):
        company.runall()

In [38]:
class Company(object):
    def __init__(self,ticker):
        self.ticker = ticker
        self.financials = YahooFinancials(self.ticker)
        
    # Gather all possible quantitative metrics available    
    def runall():
        

In [39]:
Company('aapl')

NameError: name 'YahooFinancials' is not defined